In [1]:
import mne

In [19]:
import dotenv
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from brainmdd.features.psd import EEGPowerSpectrum
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix
import mne
import enum
import joblib

# Declare Global Variables
NUM_CHANNELS = 16
MODMA_SFREQ = 250


# Declare Feature Dictionary
featureDict = {
    "norm_power": {
        "mdd": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            },
        "control": {
            "delta": [],
            "theta": [],
            "alpha": [],
            "beta": [],
            "gamma": []
            }
        }
    }    



In [5]:
# load the epochs
testHCEpochsFilePath = "/Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/hc1610epochs.fif"
testMDDEpochsFilePath = "/Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/mdd1610epochs.fif"

mdd_epochs = mne.read_epochs(testMDDEpochsFilePath, preload=True)
hc_epochs = mne.read_epochs(testHCEpochsFilePath, preload=True)

Reading /Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/mdd1610epochs.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms


        0 CTF compensation matrices available
Not setting metadata
180 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/hc1610epochs.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available
Not setting metadata
180 matching events found
No baseline correction applied
0 projection items activated


/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_3561/3411271536.py:5: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/mdd1610epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  mdd_epochs = mne.read_epochs(testMDDEpochsFilePath, preload=True)
/var/folders/kf/plh_bl0s64g1swyqvnc651pr0000gn/T/ipykernel_3561/3411271536.py:6: RuntimeWarning: This filename (/Users/wachiii/Workschii/brain-mdd/dataset/predct/eyeopen_16Channels/selected10subepochs/hc1610epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  hc_epochs = mne.read_epochs(testHCEpochsFilePath, preload=True)


In [6]:
psdObjMdd = EEGPowerSpectrum(mdd_epochs, 250, ifNormalize=False)
mddPsdFeatures = psdObjMdd.run()
psdObjHc = EEGPowerSpectrum(hc_epochs, 250, ifNormalize=False)
hcPsdFeatures = psdObjHc.run()
print("MDD PSD Features: ", mddPsdFeatures.shape)
print("HC PSD Features: ", hcPsdFeatures.shape)

MDD PSD Features:  (180, 5, 16)
HC PSD Features:  (180, 5, 16)


In [30]:
psdOnlyModelPath = "/Users/wachiii/Workschii/brain-mdd/models/CrossValidation/bandpowerCv/Random Forest_NoFeatureSelection/model_fold_1.pkl"
psdOnlyModel = joblib.load(psdOnlyModelPath)


In [31]:
mddPsdFeatures = mddPsdFeatures.reshape(mddPsdFeatures.shape[0], -1)
hcPsdFeatures = hcPsdFeatures.reshape(hcPsdFeatures.shape[0], -1)
mddFeature = np.concatenate([mddPsdFeatures], axis=1)
hcFeature = np.concatenate([hcPsdFeatures], axis=1)

X = np.concatenate([mddFeature, hcFeature], axis=0)
y = np.concatenate([np.ones(mddFeature.shape[0]), np.zeros(hcFeature.shape[0])], axis=0)
print("X shape: ", X.shape)
print("MDD PSD Features Reshape: ", mddPsdFeatures.shape)

X shape:  (360, 80)
MDD PSD Features Reshape:  (180, 80)


In [32]:
# Predict the MDD and HC
y_pred = psdOnlyModel.predict(X)
acc = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
print("Accuracy: ", acc)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)


Accuracy:  0.5444444444444444
Precision:  0.5317460317460317
Recall:  0.7444444444444445
F1 Score:  0.6203703703703703
